<a href="https://colab.research.google.com/github/chesterhuynguyen/huynguyen_INFO5731_Fall2023/blob/main/Huy_Nguyen_In_class_exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The second In-class-exercise (09/13/2023, 40 points in total)

Kindly use the provided .ipynb document to write your code or respond to the questions. Avoid generating a new file.
Execute all the cells before your final submission.

This in-class exercise is due tomorrow September 14, 2023 at 11:59 PM. No late submissions will be considered.

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data.

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
Research question: collect 1000 items under $5 on Amazon for fall decoration

Data to collect:
Name/Title
Price
Link

Detailed step:
Step 1: Define the scraping link
Step 2: set up environment  install the required libraries, such as 'urlib', 'requests' and 'beautifulsoup4'
Step 3: Obtain an Access Token: Amazon may require an access token for authentication. Follow their API documentation to obtain one.
Step 4: Define Web Scraping method and write code sniplet
Step 5: Extract data: Name, price, link
Step 6: Store Data: Save the collected data in a structured format like JSON, CSV, or a database

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys

def scrape_amazon_items(url, max_price, max_items):
    items = []
    page = 1

    try:
        while len(items) < max_items:
            current_url = f"{url}&page={page}"
            response = requests.get(current_url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                product_elements = soup.find_all('div', class_='s-result-item')
                for product_element in product_elements:
                    title_element = product_element.find('h2')
                    price_element = product_element.find('span', class_='a-price')
                    link_element = product_element.find('a', class_='a-link-normal')

                    if title_element and price_element and link_element:
                        title = title_element.text.strip()
                        price = price_element.text.strip()
                        price_numeric = float(price.replace('$', '').replace(',', ''))
                        link = 'https://www.amazon.com' + link_element['href']

                        if price_numeric <= max_price:
                            items.append({'Name': title, 'Price': price, 'Link': link})

            page += 1
    except KeyboardInterrupt:
        print("Scraping interrupted by user. Collected data will be displayed.")

    return items[:max_items]

if __name__ == "__main__":
    amazon_url = "https://www.amazon.com/s?k=fall+decorations&crid=3V5UKRT2Y3CUV&sprefix=fall+decorations%2Caps%2C142&ref=nb_sb_noss_1"
    max_price = 5.0
    max_items = 1000

    fall_decorations = scrape_amazon_items(amazon_url, max_price, max_items)

    # Create a Pandas DataFrame from the collected data
    df = pd.DataFrame(fall_decorations)

    # Print the DataFrame
    print(df)


Scraping interrupted by user. Collected data will be displayed.
Empty DataFrame
Columns: []
Index: []


Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2013-2023).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [ ]:
pip install urllib3


In [3]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Function to format Google Scholar date to a year
def parse_year(date_str):
    try:
        return datetime.strptime(date_str, '%Y').year
    except ValueError:
        return None

# Create an empty DataFrame to store the data
columns = ['Title', 'Venue', 'Year', 'Authors', 'Abstract']
df = pd.DataFrame(columns=columns)

# Number of articles to collect
num_articles_to_collect = 1000

# Start index for pagination
start_index = 0

# Define a User-Agent header
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36'

while len(df) < num_articles_to_collect:
    # Construct the search URL with the keyword and date range
    search_url = f'https://scholar.google.com/scholar?q=information+retrieval&as_ylo=2013&as_yhi=2023&start={start_index}'

    # Send a GET request to the URL using urllib with a User-Agent header
    headers = {'User-Agent': user_agent}
    req = urllib.request.Request(search_url, headers=headers)

    try:
        response = urllib.request.urlopen(req)
        html = response.read()

        # Parse the HTML content of the page
        soup = BeautifulSoup(html, 'html.parser')

        # Find the search results
        search_results = soup.find_all('div', class_='gs_ri')

        for result in search_results:
            # Extract information for each article (modify this part to fit your needs)
            title = result.find('h3', class_='gs_rt').get_text()
            venue = result.find('div', class_='gs_a').get_text()
            year = parse_year(result.find('div', class_='gs_a').get_text())
            authors = result.find('div', class_='gs_a').get_text().split('-')[0].strip()
            abstract = result.find('div', class_='gs_rs').get_text()

            # Append the article information to the DataFrame
            df = df.append({'Title': title, 'Venue': venue, 'Year': year, 'Authors': authors, 'Abstract': abstract},
                           ignore_index=True)

        # Increment the start index for pagination
        start_index += 10

        # Ensure we don't exceed the desired number of articles
        if len(df) >= num_articles_to_collect:
            break

    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e}")
        break

# Save the collected articles to a CSV file
df.to_csv('google_scholar_articles.csv', index=False)

print(f"Collected {len(df)} articles and saved to 'google_scholar_articles.csv'.")


<ipython-input-3-72aded150473>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'Venue': venue, 'Year': year, 'Authors': authors, 'Abstract': abstract},
<ipython-input-3-72aded150473>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'Venue': venue, 'Year': year, 'Authors': authors, 'Abstract': abstract},
<ipython-input-3-72aded150473>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'Venue': venue, 'Year': year, 'Authors': authors, 'Abstract': abstract},
<ipython-input-3-72aded150473>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

HTTP Error: HTTP Error 429: Too Many Requests
Collected 600 articles and saved to 'google_scholar_articles.csv'.


Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data.

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text

In [ ]:
#collect 1000 posts on instagram with #sunday hashtag from yesterday

import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Initialize session
session = requests.Session()
session.headers.update({'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"})

# Set the hashtag and the number of posts to collect
hashtag = 'sunday'
num_posts_to_collect = 1000

# Calculate the date for yesterday
yesterday = datetime.now() - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# Initialize lists to store data
usernames = []
posted_times = []
texts = []

# Instagram URL for hashtag search
base_url = 'https://www.instagram.com/explore/tags/'
url = f'{base_url}{hashtag}/'

# Send a GET request to the URL
response = session.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract post data
    posts = soup.find_all('article')

    for post in posts:
        # Extract user name
        username = post.find('a', class_='sqdOP').text

        # Extract posted time
        timestamp = post.find('time')['datetime']

        # Extract post text
        text = post.find('div', class_='C4VMK').span.text

        # Append data to lists
        usernames.append(username)
        posted_times.append(timestamp)
        texts.append(text)

        # Check if we've collected enough posts
        if len(usernames) >= num_posts_to_collect:
            break

# Create a Pandas DataFrame (if you want) and print the data
import pandas as pd

data = {'User_name': usernames, 'Posted_time': posted_times, 'Text': texts}
df = pd.DataFrame(data)

print(df)





Empty DataFrame
Columns: [User_name, Posted_time, Text]
Index: []
